In [1]:
import cv2
import numpy as np
from keras_facenet import FaceNet
from pymongo import MongoClient
from sklearn.metrics.pairwise import cosine_similarity

KeyboardInterrupt: 

In [3]:
# Initialize FaceNet model
model = FaceNet()

# Connect to MongoDB
client = MongoClient("mongodb://localhost:27017/")
db = client["biometric_db"]
users_collection = db["users"]

In [4]:
# Face detection using OpenCV (Haar Cascades)
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

In [5]:
def register_user(username):
    cap = cv2.VideoCapture(0)
    embeddings = []
    
    print("Starting registration. Please hold steady for each capture.")
    frame_count = 0

    while frame_count < 5:  # Capture 5 frames
        ret, frame = cap.read()
        if not ret:
            print("Failed to capture frame. Check the webcam connection.")
            break
        
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.1, 4)

        for (x, y, w, h) in faces:
            face = frame[y:y+h, x:x+w]
            face_embedding = model.embeddings([face])[0]
            embeddings.append(face_embedding)
            frame_count += 1
            
            # Draw a rectangle around the face and show the frame
            cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
            cv2.imshow("Registering Face", frame)

            # Wait for 1 second before capturing the next frame
            time.sleep(1)
            print(f"Captured frame {frame_count} of 5")
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    cap.release()
    cv2.destroyAllWindows()
    
    if frame_count == 5:
        user_data = {"username": username, "embeddings": embeddings}
        users_collection.insert_one(user_data)
        print(f"User '{username}' registered successfully with 5 frames!")
    else:
        print("Registration failed. Please try again.")

In [6]:
def authenticate_user():
    cap = cv2.VideoCapture(0)
    ret, frame = cap.read()
    cap.release()
    cv2.destroyAllWindows()
    
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.1, 4)
    
    if len(faces) == 0:
        print("No face detected!")
        return None
    
    # Extract embedding for the detected face
    for (x, y, w, h) in faces:
        face = frame[y:y+h, x:x+w]
        test_embedding = model.embeddings([face])[0]
        
        # Fetch all users from database
        all_users = users_collection.find()
        
        # Iterate through each user's embeddings and check for a match
        for user in all_users:
            stored_embeddings = user["embeddings"]
            similarities = [cosine_similarity([stored], [test_embedding])[0][0] for stored in stored_embeddings]
            
            if any(similarity > 0.9 for similarity in similarities):  # Threshold of 0.9
                print(f"Authenticated: {user['username']}")
                return user['username']
    
    print("Authentication failed.")
    return None


In [7]:
def update_user_embeddings(username, new_embedding):
    users_collection.update_one(
        {"username": username},
        {"$push": {"embeddings": new_embedding}}
    )
    print(f"Embeddings for '{username}' updated successfully!")


In [1]:
def main():
    while True:
        choice = input("Choose an option:\n1. Register\n2. Authenticate\n3. Exit\n")
        
        if choice == '1':
            username = input("Enter username: ")
            register_user(username)
        
        elif choice == '2':
            username = authenticate_user()
            if username:
                print(f"Welcome, {username}!")
                
                # Capture a new embedding after successful authentication
                cap = cv2.VideoCapture(0)
                ret, frame = cap.read()
                cap.release()
                cv2.destroyAllWindows()
                
                gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
                faces = face_cascade.detectMultiScale(gray, 1.1, 4)
                
                for (x, y, w, h) in faces:
                    face = frame[y:y+h, x:x+w]
                    new_embedding = model.embeddings([face])[0]
                    update_user_embeddings(username, new_embedding)
        
        elif choice == '3':
            break
        else:
            print("Invalid option. Try again.")

if __name__ == "__main__":
    main()
